In [2]:
from matplotlib import pyplot as plt
import seaborn as sns

from bs4 import BeautifulSoup
import requests

import string
import re
import random

import pandas as pd
import numpy as np

In [3]:
url = 'https://www.kommersant.ru/archive/news?from=news&page=5'

In [4]:
build_response = requests.get(url)
page_text = build_response.content
soup = BeautifulSoup(page_text, "html.parser")

#getting links out
links = []
for link in soup.find_all('a', class_='uho__link uho__link--overlay'):
    links.append('https://www.kommersant.ru' + link.get('href'))

In [5]:
texts = []
dates = []
headings = []
authors = []

for url in links:
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    souped_texts = soup.find_all("div", class_='article_text_wrapper')
    for i in souped_texts:
        texts.append(i.text.strip())
    souped_times = soup.find('time', class_='doc_header__publish_time')
    for i in souped_times:
        dates.append(i.text)
    souped_h1 = soup.find('h1')
    for i in souped_h1:
        headings.append(i.text.strip())
    souped_bst = soup.find_all()
    for i in souped_texts:
        author = i.find('p', class_='doc__text document_authors')
        authors.append(author.get_text().strip() if author else 'None')     

In [8]:
df['date_published'] = pd.to_datetime(df['date_published'], infer_datetime_format = True)
df.head()

,date_published,author,heading,summary
0,2023-03-03 15:20:00,Эрдни Кагалтынов,Мэр Енакиево в ДНР сообщил о сбитом над городо...,Мэр города Енакиево в Донецкой Народной Респуб...
1,2023-03-03 15:15:00,"Антон Голицын, Ярославль",В Ярославле «положенца» приговорили к 15 годам...,"Ярославский областной суд сегодня, 3 марта, вы..."
2,2023-03-03 15:02:00,Эрдни Кагалтынов,ТАСС: автомобиль Генпрокуратуры попал в ДТП с ...,На Кутузовском проспекте в Москве столкнулись ...
3,2023-03-03 14:58:00,Анастасия Ларина,После ночных сообщений о взрывах в Коломне неб...,"Глава Коломны Александр Гречищев сообщил, что ..."
4,2023-03-03 14:57:00,None,Песков: основная часть совещания Путина с Сове...,Пресс-секретарь президента России Дмитрий Песк...


In [9]:
translit_voc = {'а': 'a', 'б': 'b', 'в': 'v', 'г': 'g', 'д': 'd', 'е': 'e', 'ё': 'yo', 'ж': 'zh', 'з': 'z', 'и': 'i','й': 'y', 'к': 'k', 'л': 'l', 'м': 'm', 'н': 'n', 'о': 'o', 'п': 'p', 'р': 'r', 'с': 's', 'т': 't', 'у': 'u', 'ф': 'f', 'х': 'kh', 'ц': 'ts', 'ч': 'ch', 'ш': 'sh', 'щ': 'shch', 'ъ': 'j', 'ы': 'y', 'ь': 'j', 'э': 'e', 'ю': 'yu', 'я': 'ya'}


In [10]:
#transliterated example
translit = []
for i in df.author:
    i = i.lower() + ' '
    words = list(i)
    for i in words:
        if i in translit_voc.keys():
            translit.append(translit_voc[i])
        else:
            translit.append(' ')

bunch = ''.join(translit)
bunch_alt = bunch.split(' ')
bunch_alt = [i for i in bunch_alt if i]
first_names = [bunch_alt[i] for i in range(0, len(bunch_alt), 2) if i]
last_names = [bunch_alt[i] for i in range(1, len(bunch_alt), 2) if i]
last_names
prompt = first_names[0].capitalize() + ' ' + last_names[0].capitalize()
prompt

'Anton Kagaltynov'